In [1]:
!pip install Bio
!pip install pysam

import pysam
import numpy as np
import pandas as pd
from Bio import SeqIO
from scipy.stats import binom
from collections import Counter
import matplotlib.pyplot as plt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.0/281.0 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.0/22.0 MB 54.9 MB/s eta 0:00:00


In [2]:
working_dir = "/kaggle/working/"
data_dir = "/kaggle/input/squidrna/"

gfl_bam = pysam.AlignmentFile(data_dir + "gfl.bam", "rb")

In [3]:
ids = []
sequences = []
orf_starts = []
orf_ends = []

with open(working_dir + "outfile.txt", "w") as out:

    for seq in SeqIO.parse(data_dir + "pealeii.fasta", "fasta"):
    
        out.write(seq.id + "\n")
    
        ids.append(seq.id)
        sequences.append(seq.seq)
    
        desc = seq.description.split("\t")
        orf_starts.append(desc[2])
        orf_ends.append(desc[4])

df = pd.DataFrame({"id": ids, "start": orf_starts, "end": orf_ends, "seq": sequences})
df["start"] = df["start"].astype(int)
df["end"] = df["end"].astype(int)

In [4]:
i = 0
error_rate = 1e-3
min_base_quality = 30
alpha = 0.05

transcript_name = df["id"][i]
orf_start = df["start"][i]
orf_end = df["end"][i]
orf_range = orf_end - orf_start

In [5]:
gene_pileup = gfl_bam.pileup(
    start=orf_start,
    stop=orf_end,
    min_base_quality=min_base_quality
)

with open(working_dir + "modifications.txt", "w") as results:
    
    results.write("Position\tCoverage\tA\tC\tT\tG\tVar\tTotal\tPadj\n")

    for column in gene_pileup:

        base_counter = Counter({"A": 0, "C": 0, "T": 0, "G": 0})

        for read in column.pileups:
            if not read.is_del and not read.is_refskip:

                base = read.alignment.query_sequence[read.query_position]
                base_counter.update(base)

        ref, var = base_counter.most_common(2)
        pval = binom.cdf(ref[1], ref[1] + var[1], 1 - error_rate)

        if pval < alpha / orf_range:
            
            results.write(f"{column.pos}\t{column.n}\t")
            results.write(f"{base_counter['A']}\t{base_counter['C']}\t")
            results.write(f"{base_counter['T']}\t{base_counter['G']}\t")
            results.write(f"{var[1]}\t{ref[1] + var[1]}\t{pval:.2g}\n")